In [1]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import time
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [5]:
sql_course = """SELECT * FROM course"""
data_course = ml_df.get_df_from_sql(sql_course)
print(f'The shape of data_course {data_course.shape}')
sql_plan = """SELECT * FROM plan"""
data_plan = ml_df.get_df_from_sql(sql_plan)
print(f'The shape of data_plan {data_plan.shape}')
sql_radiation = """SELECT * FROM radiation"""
data_radiation = ml_df.get_df_from_sql(sql_radiation)
print(f'The shape of data_radiation {data_radiation.shape}')
sql_radiationhstry = """SELECT * FROM radiationhstry"""
data_radiationhstry = ml_df.get_df_from_sql(sql_radiationhstry)
print(f'The shape of data_radiationhstry {data_radiationhstry.shape}')
sql_patient = """SELECT * FROM patient"""
data_patient = ml_df.get_df_from_sql(sql_patient)
print(f'The shape of data_patient {data_patient.shape}')

The shape of data_course (58360, 11)
The shape of data_plan (148421, 13)
The shape of data_radiation (722368, 11)
The shape of data_radiationhstry (2742710, 12)
The shape of data_patient (51062, 2)


In [10]:
try:
    data_course.drop(columns = ['LastUpdated'], inplace = True)
    data_plan.drop('LastUpdated', axis = 1, inplace = True)
    data_radiation.drop('LastUpdated', axis = 1, inplace = True)
    data_radiationhstry.drop('LastUpdated', axis = 1, inplace = True)
    data_patient.drop('LastUpdated', axis = 1, inplace = True)
    print('Drop columns')
except:
    print('Finish droppping columns')

Drop columns


##### Radiationhstry 表里是治疗的信息（treatment information），其中包括治疗开始时间和结束时间，治疗序数，监控单元
##### Appointment 表里是预约信息（appointment information），其中包括预约开始时间和结束时间，预约的状态等
##### Radiationhstry 和 Appointment 两张表之间没有直接连接的key ，需要通过患者接受治疗的时间来连接两张表，换句话说，Appointment.ScheduledStartTime 和Radiationhstry.TreatmentStartTime 提供了每位患者预约信息（scheduled appointment）和治疗记录（treatment records）的连接

In [4]:
data_course_ = data_course.drop('AliasSerNum', axis = 1, inplace = False)
course_plan = pd.merge(data_course_, data_plan, on = 'CourseSerNum', how = 'inner')
print(f'The shape of course_plan {course_plan.shape}')

data_radiation.drop(columns = ['AliasSerNum'])
course_plan_rad = pd.merge(course_plan, data_radiation, on = 'PlanSerNum', how = 'inner')
print(f'The shape of course_plan_rad {course_plan_rad.shape}')

data_radiationhstry.drop(columns = ['AliasSerNum'])
course_plan_rad_radhstry = pd.merge(course_plan_rad, data_radiationhstry, on = 'RadiationSerNum', how = 'inner')
print(f'The shape of course_plan_rad_radhstry {course_plan_rad_radhstry.shape}')

The shape of course_plan (148421, 20)
The shape of course_plan_rad (722289, 29)
The shape of course_plan_rad_radhstry (2742565, 39)


==========================================================================

## Radiation 表

##### 只保留放射剂量大于0 的数据

In [9]:
data_radiation.AliasSerNum.value_counts()

37    722368
Name: AliasSerNum, dtype: int64

##### data_radiation 所有记录都是 AliasSerNum = 37，Core Tables from ARIA

In [10]:
data_radiation.DeliveryType.value_counts()

TREATMENT    482800
IMAGE        239513
                 55
Name: DeliveryType, dtype: int64

In [11]:
print(f'The shape of data_radiation {data_radiation.shape}')
data_radiation_TM = data_radiation[(data_radiation.DeliveryType == 'TREATMENT')]
print(f'The shape of data_radiation_TM {data_radiation_TM.shape}')
data_radiation_IM = data_radiation[(data_radiation.DeliveryType == 'IMAGE')]
print(f'The shape of data_radiation_IM {data_radiation_IM.shape}')

The shape of data_radiation (722368, 10)
The shape of data_radiation_TM (482800, 10)
The shape of data_radiation_IM (239513, 10)


##### data_radiation 的DeliveryType 有两种，分别是 Treatment 和Image，通过放射剂量筛选之后，只剩下Treatment 类型

In [12]:
data_radiation_IM.MU.value_counts()

0.000      239436
1.000          61
430.000         6
774.000         6
112.343         3
109.800         1
Name: MU, dtype: int64

##### Image 类型里，MU 和MUCoeff 几乎全都是0

##### 目前我们只使用DeliveryType = 'TREATMENT' 并且放射剂量 > 0

In [13]:
print(f'The shape of data_radiation {data_radiation.shape}')
data_radiation_ = data_radiation[data_radiation.DeliveryType == 'TREATMENT']
print(f'The shape of data_radiation_ {data_radiation_.shape}')
data_radiation_ = data_radiation_[(data_radiation_.MUCoeff > 0) & 
                                 (data_radiation_.MU > 0)]
print(f'The shape of data_radiation_ {data_radiation_.shape}')

The shape of data_radiation (722368, 10)
The shape of data_radiation_ (482800, 10)
The shape of data_radiation_ (243540, 10)


In [14]:
data_radiation_grouped = data_radiation_.groupby('PlanSerNum')
data_radiation_grouped.get_group(105405)

,RadiationSerNum,RadiationAriaSer,AliasSerNum,ResourceSerNum,PlanSerNum,RadiationId,TechniqueId,MU,MUCoeff,DeliveryType
0,1,3577119,37,7,105405,1.1 ARC1,ARC,237.688,100.0,TREATMENT
1,2,3577120,37,7,105405,1.2 ARC2,ARC,257.686,100.0,TREATMENT


## Radiationhstry 表

In [15]:
data_radiationhstry.AliasSerNum.value_counts()

37    2742710
Name: AliasSerNum, dtype: int64

##### data_radiationhstry 所有记录都是 AliasSerNum = 37，Core Tables from ARIA，和data_radiation 一样

In [16]:
data_radiationhstry.RadiationHstrySerNum.unique()

array([      1,       2,       3, ..., 2742708, 2742709, 2742710],
      dtype=int64)

##### RadiationHstrySerNum，RadiationHstryAriaSer 都是惟一的

In [17]:
data_radiationhstry_grouped = data_radiationhstry.groupby('RadiationSerNum')# 574554
# data_radiationhstry_grouped.get_group(574554).sort_values(by = 'TreatmentStartTime', ascending = True)

In [18]:
data_radiationhstry_grouped.count()

,RadiationHstrySerNum,RadiationHstryAriaSer,AliasSerNum,TreatmentStartTime,TreatmentEndTime,TreatmentTime,FractionNumber,ImagesTaken,ImageDuration,UserName
RadiationSerNum,,,,,,,,,,
1,25,25,25,25,25,25,25,25,25,25
2,25,25,25,25,25,25,25,25,25,25
3,28,28,28,28,28,28,28,28,28,28
4,18,18,18,18,18,18,18,18,18,18
5,20,20,20,20,20,20,20,20,20,20
6,20,20,20,20,20,20,20,20,20,20
7,44,44,44,44,44,44,44,44,44,44
8,45,45,45,45,45,45,45,45,45,45
9,25,25,25,25,25,25,25,25,25,25


In [19]:
data_radiationhstry.RadiationSerNum.value_counts()

200240    441
360727    417
360730    365
360731    354
434811    324
435619    322
435403    312
254237    301
360732    299
360733    287
443245    274
444958    270
249405    248
445600    246
156365    240
622292    240
622296    228
622284    227
623049    220
622300    219
319609    187
319606    186
219927    180
284367    173
531770    173
574671    166
574546    166
574550    166
284370    164
574554    164
         ... 
299298      1
27666       1
31760       1
44042       1
46945       1
91995       1
46089       1
54277       1
481627      1
94042       1
67415       1
370949      1
366851      1
65384       1
34663       1
188915      1
66623       1
180727      1
70717       1
176633      1
76858       1
85046       1
5149        1
91187       1
95281       1
616342      1
103469      1
111657      1
123939      1
37867       1
Name: RadiationSerNum, Length: 260058, dtype: int64

## Plan 表

In [ ]:
data_plan.AliasSerNum.value_counts()

##### Plan 所有记录都是 AliasSerNum = 37，Core Tables from ARIA，和data_radiationhstry，data_radiation 一样

In [ ]:
data_plan.Status.value_counts()

In [ ]:
data_plan[data_plan.Status == 'Rejected']

In [ ]:
data_plan_ = data_plan[data_plan.Status != 'Unapproved']
data_plan_grouped = data_plan_.groupby('CourseSerNum')
# data_plan_grouped.get_group(1)

In [ ]:
data_plan_grouped.groups.keys()

In [ ]:
data_plan_grouped.get_group(268)

In [ ]:
data_plan.Status.iloc[0]

## Course 表

In [ ]:
data_course.AliasSerNum.value_counts()

#####  Course 所有记录都是 AliasSerNum = 37，Core Tables from ARIA，和Plan，data_radiationhstry，data_radiation 一样

==========================================================================

## 数据预处理

In [98]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [73]:
def data_compare(df1, df2, name):
    df_count1 = df1[name].value_counts().index.tolist()
    df_count2 = df2[name].value_counts().index.tolist()
    same = list(set(df_count1).intersection(set(df_count1)))
    print(f'same: {len(same)}')
    
    a = list(set(df_count1).difference(set(df_count2)))
    print(f'df1 has, df2 does not has: {len(a)}')
    b = list(set(df_count2).difference(set(df_count1)))
    print(f'df2 has, df1 does not has: {len(b)}')

In [77]:
data_compare(co_pa, ra_rh_pl, 'AliasSerNum')

same: 1
df1 has, df2 does not has: 0
df2 has, df1 does not has: 0


In [78]:
# Radiation 的预处理
print('='*20)
print('Process data_radiation')
print(f'\nThe shape of data_radiation {data_radiation.shape}')
data_radiation_ = data_radiation[(data_radiation.DeliveryType == 'TREATMENT') &
                                 (data_radiation.MU > 0) &
                                 (data_radiation.MUCoeff > 0)]
print(f'The shape of data_radiation_ {data_radiation_.shape}')

# Radiationhstry 的预处理
print('\nProcess data_radiationhstry')
print(f'\nThe shape of data_radiationhstry {data_radiationhstry.shape}')
data_radiationhstry_ = data_radiationhstry[data_radiationhstry.TreatmentStartTime > pd.Timestamp('2015-01-01 00:00:00')]
print(f'The shape of data_radiationhstry_ {data_radiationhstry_.shape}')

print(f'\nMerge data_radiation_ and data_radiationhstry_')
ra_rh = pd.merge(data_radiation_, data_radiationhstry_, 
                 on = ['RadiationSerNum', 'AliasSerNum'], how = 'inner')
print(f'\nThe shape of ra_rh {ra_rh.shape}')

print('='*20)
print(f'Merge data_plan and ra_rh')
print(f'\nThe shape of data_plan {data_plan.shape}')
ra_rh_pl = pd.merge(data_plan, ra_rh, on = ['PlanSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of ra_rh_pl {ra_rh_pl.shape}')

print('='*20)
print(f'Merge data_course and data_patient')
print(f'\nThe shape of data_course {data_course.shape}')
print(f'The shape of data_patient {data_patient.shape}')
co_pa = pd.merge(data_course, data_patient, on = 'PatientSerNum', how = 'inner')
print(f'The shape of co_pa {co_pa.shape}')

print('='*20)
print(f'Merge co_pa and ra_rh_pl')
print(f'\nThe shape of co_pa {co_pa.shape}')
print(f'The shape of ra_rh_pl {ra_rh_pl.shape}')
co_pa_ra_rh_pl = pd.merge(co_pa, ra_rh_pl, 
                          on = ['CourseSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of co_pa_ra_rh_pl {co_pa_ra_rh_pl.shape}')

#　删除值完全相同的列
print('='*20)
print(f'Drop columns with same values')
for col in co_pa_ra_rh_pl.columns:
    if len(co_pa_ra_rh_pl[col].unique()) == 1:
        co_pa_ra_rh_pl.drop(col, axis = 1, inplace = True)
print(f'\nco_pa_ra_rh_pl shape {co_pa_ra_rh_pl.shape}')

Process data_radiation

The shape of data_radiation (722368, 10)
The shape of data_radiation_ (243540, 10)

Process data_radiationhstry

The shape of data_radiationhstry (2742710, 11)
The shape of data_radiationhstry_ (592728, 11)

Merge data_radiation_ and data_radiationhstry_

The shape of ra_rh (507969, 19)
Merge data_plan and ra_rh

The shape of data_plan (148421, 12)
The shape of ra_rh_pl (507969, 29)
Merge data_course and data_patient

The shape of data_course (58360, 10)
The shape of data_patient (51062, 1)
The shape of co_pa (58348, 10)
Merge co_pa and ra_rh_pl

The shape of co_pa (58348, 10)
The shape of ra_rh_pl (507969, 29)
The shape of co_pa_ra_rh_pl (507863, 37)
Drop columns with same values

co_pa_ra_rh_pl shape (507863, 33)


In [96]:
feature_columns = [
    'RadiationHstryAriaSer', 'TreatmentStartTime', 'TreatmentEndTime', 'FractionNumber', 'ImagesTaken', 'UserName',
    'RadiationSerNum', 'RadiationId', 'ResourceSerNum', 'MU', 'MUCoeff', 
    'PatientSerNum',
    'CourseId'
]

data_part2 = co_pa_ra_rh_pl[feature_columns]
data_part2['date'] = data_part2.apply(lambda x: x.TreatmentStartTime.strftime("%Y--%m--%d"), axis = 1)
data_part2.sort_values(by = ['PatientSerNum', 'RadiationHstryAriaSer'], inplace = True)

C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [97]:
data_part2_grouped = data_part2.groupby('PatientSerNum')
data_part2_grouped.get_group(83)

,RadiationHstryAriaSer,TreatmentStartTime,TreatmentEndTime,FractionNumber,ImagesTaken,UserName,RadiationSerNum,RadiationId,ResourceSerNum,MU,MUCoeff,PatientSerNum,CourseId,date
0,3679490,2018-06-14 14:53:00,2018-06-14 14:53:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,83,C1,2018--06--14
1,3679491,2018-06-14 14:53:00,2018-06-14 14:53:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,83,C1,2018--06--14
3,3679492,2018-06-14 14:54:00,2018-06-14 14:54:00,1,0,rislam,632258,1.2 POST,177,434.852,109.881,83,C1,2018--06--14
4,3679493,2018-06-14 14:55:00,2018-06-14 14:56:00,1,0,rislam,632258,1.2 POST,177,434.852,109.881,83,C1,2018--06--14
2,3679494,2018-06-14 14:57:00,2018-06-14 14:57:00,1,0,rislam,632257,1.1 ANT,177,432.523,109.293,83,C1,2018--06--14


In [99]:
data_part2.to_csv(data_path + 'data_part2.csv')